# CSI4142 - Data Science

## Assignment 3 - Predictive analysis - Regression and Classification

Shacha Parker 300235525

Callum Frodsham 300199446

This part of the assignment takes a dataset and performs an empirical study with a linear regression task.

#### Execution of the Notebook
 
FILL THIS IN WHEN FINALIZED

### Dataset Information
Dataset name: 

<a href="https://www.kaggle.com/datasets/mirichoi0218/insurance">Medical Cost Personal Datasets</a>
<br>
Provider: Miri Choi on Kaggle

<b>Features</b>

    age: age of primary beneficiary - quantitative - continuous

    sex: insurance contractor gender, female, male - categorical, ordinal

    bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9 - quantitative - continuous

    children: Number of children covered by health insurance / Number of dependents - quantitative, discrete

    smoker: does the beneficiary smoke - categorical, nominal

    region: the beneficiary's residential area in the US: northeast, southeast, southwest, northwest. - categorical, nominal

    charges: Individual medical costs billed by health insurance - quantitative, continuous


In [ ]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
# CHANGE THIS TO GITHUB RAW
data = pd.read_csv('insurance.csv')

# output settings for debugging
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Data Cleaning
The dataset is already clean. This can be seen with the results of the code cell below: no entries are null, and all values are appropriate. No cleaning is needed.

In [ ]:
# Display information about the dataframe
print(data.info())

# Describe the dataframe
print(data.describe())

# Check for missing values
missing_values = data.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Check for unique values in categorical columns to identify any unclean data
categorical_columns = data.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\nUnique values in column '{column}':\n", data[column].unique())

## Categorical Feature Encoding
Through the get_dummies function, we can create the dataset with all categorical features becoming one-hot vectors. The rest of the features remain as they were.

In [ ]:
# One-hot encode the specified categorical features
data = pd.get_dummies(data, columns=['sex', 'smoker', 'region'], drop_first=False)

## EDA and Outlier detection
a. Outlier detection

In [ ]:
# Comparison graph for sex_male against sex_female
plt.figure(figsize=(10, 5))
sns.countplot(data=data, x='sex_male')
plt.title('Count of Male vs Female')
plt.xlabel('Sex (0: Female, 1: Male)')
plt.ylabel('Count')
plt.show()

# Comparison graph for smoker counts
plt.figure(figsize=(10, 5))
sns.countplot(data=data, x='smoker_yes')
plt.title('Count of Smokers vs Non-Smokers')
plt.xlabel('Smoker (0: No, 1: Yes)')
plt.ylabel('Count')
plt.show()

# Comparison for regions
plt.figure(figsize=(10, 5))
region_columns = ['region_northeast', 'region_northwest', 'region_southeast', 'region_southwest']
region_counts = data[region_columns].sum()
sns.barplot(x=region_counts.index, y=region_counts.values)
plt.title('Count of Regions')
plt.xlabel('Region')
plt.ylabel('Count')
plt.show()

Naturally, outliers don't occur on categorical data like this.

In [ ]:
# Visualize numerical features
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns
for column in numerical_columns:
    plt.figure(figsize=(10, 5))
    sns.boxplot(data=data, x=column)
    plt.title(f'Box plot of {column}')
    plt.show()

Since feature 'bmi' has only 9 outliers out of the 1000+ entries (0.67%), we can safely remove them without there being a significant statistical impact.

In [ ]:
data_without_outlier_removal = data
original = data.shape

# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data['bmi'].quantile(0.25)
Q3 = data['bmi'].quantile(0.75)

IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the outliers
data = data[(data['bmi'] >= lower_bound) & (data['bmi'] <= upper_bound)]

plt.figure(figsize=(10, 5))
sns.boxplot(data=data, x=data['bmi'])
plt.title(f'Box plot of BMI after outlier removal')
plt.show()

print("Original data shape:", original)
print("Data shape after removing outliers:", data.shape)

For the 'charges' outliers, it would negatively impact the dataset if all rows with 'charges' outliers were removed. Instead, we'll use  imputation to apply known values over the existing outlier values.

In [ ]:

# Calculate Q1 (25th percentile) and Q3 (75th percentile) for 'charges'
Q1_charges = data['charges'].quantile(0.25)
Q3_charges = data['charges'].quantile(0.75)
IQR_charges = Q3_charges - Q1_charges
lower_bound_charges = Q1_charges - 1.5 * IQR_charges
upper_bound_charges = Q3_charges + 1.5 * IQR_charges

# Create D1 by removing rows where 'charges' is an outlier
D1 = data[(data['charges'] >= lower_bound_charges) & (data['charges'] <= upper_bound_charges)]

# Create D2 by removing 'charges' outliers but keeping the rest of the data
D2 = data.copy()
D2.loc[(D2['charges'] < lower_bound_charges) | (D2['charges'] > upper_bound_charges), 'charges'] = np.nan

# Display the shapes of the original, D1, and D2 datasets
print("Original data shape:", data.shape)
print("D1 data shape (without 'charges' outliers):", D1.shape)
print("D2 data shape (with 'charges' outliers removed):", D2.shape)

missing_values_d2 = D2['charges'].isnull().sum()
print("Missing values in each column of D2:\n", missing_values_d2)

# Get the indices of missing values in D2
missing_indices = D2[D2['charges'].isnull()].index

'''
# Randomly sample values from D1['charges'] to fill the missing values in D2
imputed_values = D1['charges'].sample(n=len(missing_indices), replace=True).values

# Impute the missing values in D2
D2.loc[missing_indices, 'charges'] = imputed_values

missing_values_d2 = D2['charges'].isnull().sum()
print("Missing values in each column of D2 after RSI:\n", missing_values_d2)

data = D2
'''

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

# Separate the features and target variable in the original data
X_original = data.drop(columns=['charges'])
y_original = data['charges']

# Train a linear regression model on the original data
imputation_model = LinearRegression()
imputation_model.fit(X_original, y_original)

# Identify the rows in D2 with missing 'charges' values
missing_indices = D2[D2['charges'].isnull()].index

# Separate the features for the rows with missing 'charges' in D2
X_missing = D2.loc[missing_indices].drop(columns=['charges'])

# Predict the missing 'charges' values using the trained model
imputed_values = imputation_model.predict(X_missing)

# Impute the missing values in D2
D2.loc[missing_indices, 'charges'] = imputed_values

# Verify that there are no more missing values in 'charges' column of D2
missing_values_d2_after_imputation = D2['charges'].isnull().sum()
print("Missing values in 'charges' column of D2 after imputation:", missing_values_d2_after_imputation)

# Update the main data variable to use D2 with imputed values
data = D2

## Feature Engineering
Before we proceed with feature engineering, we'll create new copies of the dataset with and without outlier removal that will not have the extra features.

In [ ]:
data_without_outlier_removal_or_FE = data_without_outlier_removal
data_without_FE = data

For the first new feature, we will create the age-bmi column. The age-bmi value is a row's age multiplied by its bmi, which is then normalized by dividing this number by the median age times the median bmi.

In [ ]:
medianage = data['age'].median()
medianbmi = data['bmi'].median()
data['age-bmi'] = (data['age'] * data['bmi']) / (medianage * medianbmi)
data_without_outlier_removal_with_FE = data_without_outlier_removal
data_without_outlier_removal_with_FE['age-bmi'] = (data_without_outlier_removal['age'] * data_without_outlier_removal['bmi']) / (medianage * medianbmi)

Secondly, we'll create a feature called "childandsmoker" which is a boolean value that is true if the row has a positive number of children and 'smoker' is 'yes'.

In [ ]:
# find missing values in data_without_outlier_removal_with_FE
missing_values = data_without_outlier_removal_with_FE.isnull().sum()
print(missing_values)

In [ ]:
data['childandsmoker'] = np.where((data['smoker_yes'] == True) & (data['children'] > 0), True, False)
#data_without_outlier_removal_with_FE = data_without_outlier_removal_with_FE.reindex(data.index)
data_without_outlier_removal_with_FE['childandsmoker'] = np.where((data_without_outlier_removal_with_FE['smoker_yes'] == True) & (data_without_outlier_removal_with_FE['children'] > 0), True, False)

In [ ]:
# Renaming these variables to be more legible
data_with_both = data
data_with_neither = data_without_outlier_removal_or_FE
data_with_FE = data_without_outlier_removal_with_FE
data_with_OR = data_without_FE

At this point, we have:

*data_with_both*: the data with both outliers removed and new features engineered.

*data_with_neither*: data with neither outliers removed nor feature engineering

*data_with_FE*: data including outliers without feature engineering applied

*data_with_OR*: data with outliers removed but without feature engineering

## Empirical study

First, we split each dataset into train, validation, and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data_with_both dataset
data_with_both_train, data_with_both_temp = train_test_split(data_with_both, test_size=0.3, random_state=42)
data_with_both_validation, data_with_both_test = train_test_split(data_with_both_temp, test_size=0.5, random_state=42)

# Split the data_with_neither dataset
data_with_neither_train, data_with_neither_temp = train_test_split(data_with_neither, test_size=0.3, random_state=42)
data_with_neither_validation, data_with_neither_test = train_test_split(data_with_neither_temp, test_size=0.5, random_state=42)

# Split the data_with_FE dataset
data_with_FE_train, data_with_FE_temp = train_test_split(data_with_FE, test_size=0.3, random_state=42)
data_with_FE_validation, data_with_FE_test = train_test_split(data_with_FE_temp, test_size=0.5, random_state=42)

# Split the data_with_OR dataset
data_with_OR_train, data_with_OR_temp = train_test_split(data_with_OR, test_size=0.3, random_state=42)
data_with_OR_validation, data_with_OR_test = train_test_split(data_with_OR_temp, test_size=0.5, random_state=42)

In [ ]:
# Prepare the data
X_train = data_with_neither_train.drop(columns=['charges'])
y_train = data_with_neither_train['charges']
X_val = data_with_neither_validation.drop(columns=['charges'])
y_val = data_with_neither_validation['charges']

# Initialize the linear regression model
model = LinearRegression()

# Perform 4-fold cross-validation on the training set
kf = KFold(n_splits=4, shuffle=True, random_state=42)
mse_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Calculate MSE and R2 on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

# Print the results
print("Cross-validated MSE scores:", -mse_scores)
print("Cross-validated R2 scores:", r2_scores)
print("Mean cross-validated MSE:", -mse_scores.mean())
print("Mean cross-validated R2:", r2_scores.mean())
print("Validation MSE:", mse_val)
print("Validation R2:", r2_val)

In [ ]:
# Assign scores to variables for data with neither
mse_scores_neither = mse_scores
r2_scores_neither = r2_scores
mse_val_neither = mse_val
r2_val_neither = r2_val

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_train = data_with_OR_train.drop(columns=['charges'])
y_train = data_with_OR_train['charges']
X_val = data_with_OR_validation.drop(columns=['charges'])
y_val = data_with_OR_validation['charges']

# Initialize the linear regression model
model = LinearRegression()

# Perform 4-fold cross-validation on the training set
kf = KFold(n_splits=4, shuffle=True, random_state=42)
mse_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Calculate MSE and R2 on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

# Print the results
print("Cross-validated MSE scores:", -mse_scores)
print("Cross-validated R2 scores:", r2_scores)
print("Mean cross-validated MSE:", -mse_scores.mean())
print("Mean cross-validated R2:", r2_scores.mean())
print("Validation MSE:", mse_val)
print("Validation R2:", r2_val)

In [ ]:
# Assign scores to variables for data with outliers removed (with_OR)
mse_scores_with_OR = mse_scores
r2_scores_with_OR = r2_scores
mse_val_with_OR = mse_val
r2_val_with_OR = r2_val

In [ ]:
X_train = data_with_FE_train.drop(columns=['charges'])
y_train = data_with_FE_train['charges']
X_val = data_with_FE_validation.drop(columns=['charges'])
y_val = data_with_FE_validation['charges']

# Initialize the linear regression model
model = LinearRegression()

# Perform 4-fold cross-validation on the training set
kf = KFold(n_splits=4, shuffle=True, random_state=42)
mse_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Calculate MSE and R2 on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

# Print the results
print("Cross-validated MSE scores:", -mse_scores)
print("Cross-validated R2 scores:", r2_scores)
print("Mean cross-validated MSE:", -mse_scores.mean())
print("Mean cross-validated R2:", r2_scores.mean())
print("Validation MSE:", mse_val)
print("Validation R2:", r2_val)

In [ ]:
# Assign scores to variables for data with feature engineering (with_FE)
mse_scores_with_FE = mse_scores
r2_scores_with_FE = r2_scores
mse_val_with_FE = mse_val
r2_val_with_FE = r2_val

In [ ]:
X_train = data_with_both_train.drop(columns=['charges'])
y_train = data_with_both_train['charges']
X_val = data_with_both_validation.drop(columns=['charges'])
y_val = data_with_both_validation['charges']

# Initialize the linear regression model
model = LinearRegression()

# Perform 4-fold cross-validation on the training set
kf = KFold(n_splits=4, shuffle=True, random_state=42)
mse_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = model.predict(X_val)

# Calculate MSE and R2 on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

# Print the results
print("Cross-validated MSE scores:", -mse_scores)
print("Cross-validated R2 scores:", r2_scores)
print("Mean cross-validated MSE:", -mse_scores.mean())
print("Mean cross-validated R2:", r2_scores.mean())
print("Validation MSE:", mse_val)
print("Validation R2:", r2_val)

In [ ]:
# Assign scores to variables for data with both outliers removed and feature engineering (with_both)
mse_scores_with_both = mse_scores
r2_scores_with_both = r2_scores
mse_val_with_both = mse_val
r2_val_with_both = r2_val

# Display all result variables
print("MSE scores for data with neither:", mse_scores_neither)
print("R2 scores for data with neither:", r2_scores_neither)
print("Validation MSE for data with neither:", mse_val_neither)
print("Validation R2 for data with neither:", r2_val_neither)

print("\nMSE scores for data with outliers removed (with_OR):", mse_scores_with_OR)
print("R2 scores for data with outliers removed (with_OR):", r2_scores_with_OR)
print("Validation MSE for data with outliers removed (with_OR):", mse_val_with_OR)
print("Validation R2 for data with outliers removed (with_OR):", r2_val_with_OR)

print("\nMSE scores for data with feature engineering (with_FE):", mse_scores_with_FE)
print("R2 scores for data with feature engineering (with_FE):", r2_scores_with_FE)
print("Validation MSE for data with feature engineering (with_FE):", mse_val_with_FE)
print("Validation R2 for data with feature engineering (with_FE):", r2_val_with_FE)

print("\nMSE scores for data with both outliers removed and feature engineering (with_both):", mse_scores_with_both)
print("R2 scores for data with both outliers removed and feature engineering (with_both):", r2_scores_with_both)
print("Validation MSE for data with both outliers removed and feature engineering (with_both):", mse_val_with_both)
print("Validation R2 for data with both outliers removed and feature engineering (with_both):", r2_val_with_both)